In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import datetime
import lxml.etree as ET
from xml.dom import minidom
%matplotlib inline

## Import processed data 

In [ ]:
df_all = pd.read_hdf("processed_data/arxiv-oai-af.h5", key='df')
df_ml = pd.read_hdf("processed_data/arxiv-oai-af-anyml.h5", key='df')

In [ ]:
df_all[0:2].transpose()

In [ ]:
def list_to_string(l):
    return ', '.join(l)

In [ ]:
df_all.abstract = df_all.abstract.apply(str.replace,args=("\n"," "))
df_all.title = df_all.title.apply(str.replace,args=("\n"," "))

In [ ]:
df_all['categories_str'] = df_all.categories.apply(list_to_string)

In [ ]:
categories = []

for row in df_all.iterrows():
    for cat in row[1].categories:
        #print(cat)
        if cat not in categories:
            categories.append(cat)
      

In [ ]:
categories.sort()
len(categories)

In [ ]:
df_dict = {}

for cat in categories:
    df_dict[cat] = pd.DataFrame(columns=df_all.columns)

In [ ]:
count = 0
        
for row in df_all.iterrows():
    for cat in row[1].categories:
        df_dict[cat] = df_dict[cat].append(row[1])
        
    if count % 25000 is 0:
        print(count)
    count += 1

In [ ]:
df_dict_generic = df_dict.copy()

In [ ]:
for cat in categories:
    df_dict_generic[cat].drop(labels=['anymlcat', 'cs.AI', 'cs.CL', 'cs.LG', 'cs.SI', 'stat.ML'],
                              axis=1, inplace=True)

In [ ]:
for cat in categories:
    print(cat, len(df_dict_generic[cat]))

In [ ]:
!rm processed_data/arxiv_oaiaf_catkey.h5

In [ ]:
for cat in categories:
    fn = "processed_data/per_category/" + cat
    df_dict_generic[cat].to_csv(fn+".tsv", sep="\t", mode="w")
    df_dict_generic[cat].to_hdf(fn+".h5", key="df", mode="w")
    cat_key = cat.replace(".","_")
    cat_key = cat_key.replace("-","_")
    df_dict_generic[cat].to_hdf("processed_data/arxiv_oaiaf_catkey.h5", key=cat_key, mode="a")
    

In [ ]:
h = pd.HDFStore("processed_data/arxiv_oaiaf_catkey.h5")
h.physics_bio_ph

In [ ]:
for cat in ['cs.AI', 'cs.CL', 'cs.LG', 'cs.SI', 'stat.ML']:
    print(len(df_dict[cat]))